# **The generator class below generates batches of data based on following expectations:**

*   The aim with the generator is to enable utilization of more than one core during the training of an Machine Learning (ML) model while still guaranteeing that each training example (= each sequence) is only trained once during 1 epoch.
*   The generator is supposed to generate sequences from 2 different sources (csv files) for each batch. The idea behind this is that the data being generated is supposed to be fed into a Sequence Model architecture that resembels Siamese Networks. The way it should be thought is that there will be 2 identical RNN networks. One of the generated sequences in each batch will be fed into one of the RNNs and the other sequence will be fed into the second RNN. And at the end of the RNNs, 2 outputs are supposed to be concatenated and processed further depending on the type of the problem being solved by the ML model.
*  The generator is supposed to return 3 numpy arrays as (source1_sequence, source2_sequence, label).


If still not clear, the code below will hopefully make it clearer.


In [0]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from tensorflow import keras as Keras

# Only needed when running the code on Colab
from google.colab import files
import io

In [2]:
# Upload the files to Colab
csv_files = files.upload()

Saving five.csv to five.csv
Saving four.csv to four.csv
Saving one.csv to one.csv
Saving six.csv to six.csv
Saving three.csv to three.csv
Saving two.csv to two.csv


In [3]:
# Create a list of files that will be used to generate batches from.
file_path_list = []

for key, value in csv_files.items():
  file_path_list.append(key)
  
file_path_list

['five.csv', 'four.csv', 'one.csv', 'six.csv', 'three.csv', 'two.csv']

In [14]:
# Let's check the content of files. 
for f in file_path_list:
  print(f, "\n", pd.read_csv(f), "\n")

five.csv 
    col1  col2   col3  col4  col5  col6
0     1  five    one   500   500     0
1     2  five    two   501   501     0
2     3  five  three   502   502     2
3     4  five   four   503   503     1
4     5  five    six   504   504     0 

four.csv 
    col1  col2   col3  col4  col5  col6
0     1  four    one   400   400     2
1     2  four    two   401   401     2
2     3  four  three   402   402     0
3     4  four   five   403   403     1
4     5  four    six   404   404     2 

one.csv 
    col1 col2   col3  col4  col5  col6
0     1  one    two   100   100     0
1     2  one  three   101   101     1
2     3  one   four   102   102     2
3     4  one   five   103   103     0
4     5  one    six   104   104     1 

six.csv 
    col1 col2   col3  col4  col5  col6
0     1  six    one   600   600     1
1     2  six    two   601   601     1
2     3  six  three   602   602     1
3     4  six   four   603   603     2
4     5  six   five   604   604     0 

three.csv 
    col1   col2

At this point, let's define how our batches are expected to look:

Assumptions:
- Let's just look at a 1 batch with 1 single train example in it.
- In each file, columns 2 and 3 indicate the names of the sources. 
- Column 6 gives the label.
- Each row includes features for each timestep. Columns 2,3,6 need to be removed when preparing batch data.
- Assume that sequence_length is 2.
- If we start from one.csv, we will need to fetch 2 timesteps (rows) and it will be the following (before removing the columns 2,3,6):
    [1, one, two, 100, 100, 0]
    
    [2, one, three, 101, 101, 1]
- Label of the batch will be the last column value from the last timestep, whch is "1" in this case.
- Above, column 2 is giving the source1 (which is "one") here. Column 3 gives the source2, which is "three". As you see, there are 2 different source2 values here: "two" and "three". In this project, the source2 of the last timestep is taken as reference when determining the second source to fetch the data from, which is "three" in this exmaple.
- And we fetch the same rows (timesteps) from three.csv as well as following:

  [1, three, one, 300, 300, 1]

  [2, three, two, 301, 301, 1]
  
- Since we now know which rows/sources will be used for our batch, we now can remove unnecessary columns. It will look like the following:
- For the sequence from one.csv:

    [1, 100, 100]
    
    [2, 101, 101]
 
 - For the sequence from three.csv:
 
  [1, 300, 300]

  [2, 301, 301]
  
- And that's it. The whole batch will look like the following:

      [array([[[  1., 100., 100.],  [  2., 101., 101.]]], 
       array([[[  1., 300., 300.],  [  2., 301., 301.]]])], 
       array([[0., 1., 0.])
       
 which essentially is: ( [sequence_from_source1,  sequence_from_source2],  label )      
 
 Maybe worth to mention is that the label is in the form of a one-hot array to be able to use in with softmax activation.

In [0]:
class DataGenerator(Keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, file_path_list, nr_header_rows, nr_columns, sequence_length, 
                 batch_size, total_nr_timesteps_per_csv_file, nr_classes, columns_to_remove, shuffle):
        'Initialization'
        self.nr_header_rows = nr_header_rows
        self.nr_columns = nr_columns
        self.sequence_length = sequence_length
        self.nr_features = nr_columns - len(columns_to_remove)
        self.total_nr_timesteps_per_csv_file = total_nr_timesteps_per_csv_file
        self.total_nr_sequences_per_csv_file = self.total_nr_timesteps_per_csv_file - self.sequence_length + 1
        self.total_nr_sequences_in_dataset = len(file_path_list) * self.total_nr_sequences_per_csv_file
        # If given batch size is bigger than max nr of examples, set the batch size to max nr examples.
        self.batch_size = min(batch_size, self.total_nr_sequences_in_dataset)
        self.nr_classes = nr_classes
        self.columns_to_remove = columns_to_remove
        self.shuffle = shuffle
        self.prediction_timesteps = np.arange(start = self.sequence_length, 
                                          stop  = self.total_nr_timesteps_per_csv_file + 1, 
                                          step  = 1, 
                                          dtype = np.int32)

        self.file_path_list = file_path_list
        self.file_path_dict = {}
        for csv_file in file_path_list:
          self.file_path_dict[csv_file[:-4]] = csv_file
        
        self.on_epoch_end()

        
    # triggered once at the very beginning as well as at the end of each epoch
    def on_epoch_end(self):
        'Updates after each epoch'
        if self.shuffle == True:
          np.random.shuffle(self.prediction_timesteps)
          np.random.shuffle(self.file_path_list)
          
    'this function will help Keras to control how many epochs the model has run so far..'
    def __len__(self):
        'Denotes the total number of batches per epoch'
        'Note that we round up the division result with np.ceil(..)'    
        return int(np.ceil(self.total_nr_sequences_in_dataset / self.batch_size))

    'Returns a complete batch'
    def __getitem__(self, index):
        'Generate one batch of data'
        
        # which file to start reading data from. int(..) rounds down the result of division
        # '% ...' added at the end in case the division becomes equal to 'len(self.file_path_list)'
        file_index = int((index * self.batch_size) / self.total_nr_sequences_per_csv_file) % len(self.file_path_list)
        # which sample to start reading data from in the file
        prediction_timestep_index = (index * self.batch_size) % self.total_nr_sequences_per_csv_file
        
        (X, y) = self.__data_generation(file_index, prediction_timestep_index)

        return X, y

    'Actual function that generates data fom csv files'
    def __data_generation(self, file_index, prediction_timestep_index):
      
      source1_batch_data_array = np.empty((0, self.sequence_length, self.nr_features))
      source2_batch_data_array = np.empty((0, self.sequence_length, self.nr_features))
      label_array = []      
      
      for batch_idx in range(self.batch_size):
        print('batch_idx   ', batch_idx)
        df_source1 = pd.read_csv(self.file_path_list[file_index], 
                              header    = None, 
                              skiprows  = self.nr_header_rows + self.prediction_timesteps[prediction_timestep_index] - self.sequence_length, 
                              nrows     = self.sequence_length)
        #label from the last timestep of the sequence
        label = df_source1.values[self.sequence_length-1, self.nr_columns-1] 
        label_array.append(label)
        source2 = df_source1.values[self.sequence_length-1, 2]
        source1 = df_source1.values[self.sequence_length-1, 1]
        df_source1 = df_source1.drop(columns=self.columns_to_remove, axis=1)
        print("source1: ", source1)
        print("source2: ", source2)
        print("label: ", label)
        # first dim = 1 bcz we generate 1 sequence at a time and append it to the batch array
        source1_batch_data = np.reshape(df_source1.values, (1, self.sequence_length, self.nr_features))
        source1_batch_data_array = np.append(source1_batch_data_array, source1_batch_data, axis=0)

        df_source2 = pd.read_csv(self.file_path_dict[source2],
                                  header    = None, 
                                  skiprows  = self.nr_header_rows + self.prediction_timesteps[prediction_timestep_index] - self.sequence_length, 
                                  nrows     = self.sequence_length)
        df_source2 = df_source2.drop(columns=self.columns_to_remove, axis=1)
        source2_batch_data = np.reshape(df_source2.values, (1, self.sequence_length, self.nr_features))
        source2_batch_data_array = np.append(source2_batch_data_array, source2_batch_data, axis=0)
        
        prediction_timestep_index = (prediction_timestep_index + 1) % self.total_nr_sequences_per_csv_file
        if prediction_timestep_index == 0:
          file_index = (file_index + 1) % len(self.file_path_list)
      
      #print(source1_batch_data_array.shape, source2_batch_data_array.shape, len(label_array))
      return ([source1_batch_data_array, source2_batch_data_array], Keras.utils.to_categorical(label_array, num_classes=self.nr_classes))    
    

In [0]:
initialization_params = {'file_path_list'                   : file_path_list,  # list of files to fetch data from
                         'nr_header_rows'                   : 1,        # nr of rows to ignore from the top in each csv file 
                         'nr_columns'                       : 6,        # total nr of columns in each csv file 
                         'sequence_length'                  : 2,        # sequence length of each (train) sample
                         'batch_size'                       : 2,        # nr examples to generate in each step
                         'total_nr_timesteps_per_csv_file'  : 5,        # total nr of weeks per season
                         'nr_classes'                       : 3,        # nr of classes in output layer
                         'columns_to_remove'                : [1,2,5],  # column indexes that are not part of the feature list
                         'shuffle'                          : False}

In [17]:
generator = DataGenerator(**initialization_params)
for index in range (generator.__len__()):
  print(generator.__getitem__(index))

batch_idx    0
source1:  five
source2:  two
label:  0
batch_idx    1
source1:  five
source2:  three
label:  2
([array([[[  1., 500., 500.],
        [  2., 501., 501.]],

       [[  2., 501., 501.],
        [  3., 502., 502.]]]), array([[[  1., 200., 200.],
        [  2., 201., 201.]],

       [[  2., 301., 301.],
        [  3., 302., 302.]]])], array([[1., 0., 0.],
       [0., 0., 1.]], dtype=float32))
batch_idx    0
source1:  five
source2:  four
label:  1
batch_idx    1
source1:  five
source2:  six
label:  0
([array([[[  3., 502., 502.],
        [  4., 503., 503.]],

       [[  4., 503., 503.],
        [  5., 504., 504.]]]), array([[[  3., 402., 402.],
        [  4., 403., 403.]],

       [[  4., 603., 603.],
        [  5., 604., 604.]]])], array([[0., 1., 0.],
       [1., 0., 0.]], dtype=float32))
batch_idx    0
source1:  four
source2:  two
label:  2
batch_idx    1
source1:  four
source2:  three
label:  0
([array([[[  1., 400., 400.],
        [  2., 401., 401.]],

       [[  2., 401.